In [2]:
import re
import datetime
import pandas as pd
import numpy as np
import requests
import io
import json
import random
from random import randint
from base64 import b64encode, b64decode
from django.contrib.auth.hashers import make_password
from django.core.files.base import ContentFile
from decimal import Decimal
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO
# google_sheet_url = 'https://docs.google.com/spreadsheets/d/1QxCyP3uVTvUOG7IuhOCx0V-Qp07BVoIhsHBrj8NxkKU/export?format=csv'

# sheet name:seed_distribution
#google_sheet_url = 'https://docs.google.com/spreadsheets/d/1w1oCUQrlk8ODccERVI02qDndEfcWYkMOkYdLUPPCapY/export?format=csv&sheet={Sheet2}'

google_sheet_url = 'https://docs.google.com/spreadsheets/d/11u329vAS8BmEDjN8hAsKKTA8kp4fLmx9wDZe5Awx1VE/export?format=csv&sheet={Sheet1}'
r = requests.get(google_sheet_url)
data = r.content
df = pd.read_csv(BytesIO(data))
local_index = []
df = df.fillna(0)
# df = df.head(1)
df

,no,SL NO,Distribution/Return,Agents New/Old,Date,Agent Name,Unique Code,Supervisor Name,Moblie Number,100 Grm Pkt,1 Acre
0,1,1,Distribution,Old,25-02-2022,Gopala H,A,Shivakumar,7406990508,100,100
1,2,2,Distribution,Old,25-02-2022,S Muruga Sargur,C,Siddappa R,9449203315,150,150
2,3,3,Distribution,Old,25-02-2022,M Parameshwaran Nanjangud,G,Prasad B P,9738880982,30,30
3,4,4,Distribution,Old,25-02-2022,M Parameshwaran Sargur,D,Siddappa R,9738080983,150,150
4,5,5,Distribution,Old,25-02-2022,M Parameshwaran H D Kote,F,Srinivasa G,9738080984,70,70
...,...,...,...,...,...,...,...,...,...,...,...
137,138,164,Distribution,Old,27-04-2022,Mahesh K,A,Harish,9900348416,4,4
138,139,165,Distribution,Old,02-05-2022,R Ravi,H,Naveena,6360227141,0,0
139,140,167,Distribution,Old,02-05-2022,Shivumurthy,J,Naveena,9740126674,0,0
140,141,169,Distribution,Old,02-05-2022,Basappa K,A,Madappa B,9632034111,0,0


In [19]:
UserHierarchyMap.objects.get(season_id=4, subordinate=186).superior.id

92

In [5]:
def generate_request_code(input_combo_id):
    code_bank_obj = ComboIssueRequestCodeBank.objects.filter(input_combo_id=input_combo_id)[0]
    last_digit_code = code_bank_obj.last_digit
    new_code = last_digit_code + 1
    value = str(code_bank_obj.input_combo.name) + str(new_code).zfill(3)
    code_bank_obj.last_digit = new_code
    code_bank_obj.save()
    return value

In [3]:
df = pd.DataFrame(ComboIssueRequestCodeBank.objects.filter().values())
df

,id,last_digit,input_combo_id
0,1,113,3
1,16,61,4
2,10,35,12
3,2,18,18
4,8,29,10
5,3,50,5
6,11,79,13
7,5,21,7
8,4,39,6
9,9,10,11


In [47]:
import random

aggri_officer_ids= [82, 99]

r =  random.choice(aggri_officer_ids)
r

82

In [18]:
ComboIssueRequest.objects.filter(input_combo_id__in=[9]).delete()

(2,
 {'inputdistri.ComboIssueDeleteAgentLog': 0,
  'inputdistri.ComboIssueAgentInventoryReceipt': 0,
  'inputdistri.InputDistributionOtherStoreTransactionMap': 0,
  'inputdistri.ComboIssueRequestAgentMap': 1,
  'inputdistri.ComboIssueRequest': 1})

In [19]:
import random

season_id=4 
for index, row in df.iterrows():
    aggri_officer_ids= [82, 99]
#     village_name = row['Village Name']
   
    date_string = row['Date']
    date = pd.to_datetime(date_string).date()
       
    supervisor_id = User.objects.get(username=row['Supervisor Name']).id
    superior_id = UserHierarchyMap.objects.get(season_id=season_id, subordinate=supervisor_id).superior.id
   
    input_combo_obj = InputCombo.objects.get(id=8)
   
    combo_request_obj = ComboIssueRequest(
        request_code=generate_request_code(8),
        season_id=season_id,
        input_combo_id=8,
        quantity_in_numbers=row['100 Grm Pkt'],
        quantity_for_area=row['1 Acre'],
        expected_date=date,
        issue_rised_date=date,
        max_status_id=6,
        max_status_date=date,
        supervisor_id=User.objects.get(username=row['Supervisor Name']).id,
        senior_supervisor_id=superior_id,
        senior_supervisor_status_id=1,
        senior_supervisor_status_date=date,
        assitant_manager_id=79,
        assitant_manager_status_id=1,
        assitant_manager_status_date=date,
        agri_officer_id=random.choice(aggri_officer_ids),
        agri_officer_status_id = 1,
        agri_officer_status_date=date,
        gm_id = 78,
        gm_status_id =1,
        gm_status_date=date
    )
    combo_request_obj.save()
    if UserProfile.objects.filter(mobile=int(row['Moblie Number'])).exists():
        user_ids = UserProfile.objects.filter(mobile=int(row['Moblie Number'])).values_list('user_id')
        if UserClusterMap.objects.filter(user_id__in=user_ids, unique_code=row['Unique Code'], season_id=season_id):
            agent_id = UserClusterMap.objects.get(user_id__in=user_ids, unique_code=row['Unique Code'], season_id=season_id).user_id
            print(agent_id)
    combo_request_agent_obj = ComboIssueRequestAgentMap(combo_issue_request_id=combo_request_obj.id,
        agent_id=agent_id,
        issue_rised_date=datetime.datetime.now(),
        quantity_in_numbers=row['100 Grm Pkt'],
        max_status_id=2)
    combo_request_agent_obj.save()
    print(row['Agent code '])
    print(row['Distribution/Return'])

/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.expected_date received a naive datetime (2022-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.issue_rised_date received a naive datetime (2022-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.max_status_date received a naive datetime (2022-02-25 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/steve/Software/virtualenv/ccgb/lib/pyt

43
Distribution
58
Distribution
153
Distribution
122
Distribution
63
Distribution
173
Distribution
145
Distribution
135
Distribution
91
Distribution
110
Distribution
203
Distribution
141
Distribution
90
Distribution
202
Distribution
206
Distribution
172


/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:25.885530) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:25.901674) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:25.920162) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetim

Distribution
139
Distribution
210
Distribution
134
Distribution
142
Distribution
154
Distribution
85
Distribution
48
Distribution
123
Distribution
169
Distribution
182
Distribution
175
Distribution
176
Distribution
199
Distribution
171
Distribution
183
Distribution
65
Distribution
184
Distribution
178
Distribution
179
Distribution
170
Distribution
194
Distribution
49
Distribution
121
Distribution
72
Distribution
71
Distribution
51
Distribution
35
Distribution
60
Distribution
74
Distribution
138
Distribution
55
Distribution


/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.104234) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.119794) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1390: RuntimeWarning: DateTimeField ComboIssueRequest.expected_date received a naive datetime (2022-02-28 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s.%s received a naive datetime "
/home/steve

...

/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.310686) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.324591) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.344052) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetim

180
Distribution
173
Distribution
88
Distribution
50
Distribution
59
Distribution
73
Distribution
146
Distribution
182
Distribution
197
Distribution
56
Distribution
31
Distribution
111
Distribution
34
Distribution
165
Distribution
120
Distribution


/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.512550) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.530406) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.544405) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetim

32
Distribution
144
Distribution
173
Distribution
73
Distribution
88
Distribution
135
Distribution
184
Distribution
202
Distribution
32
Distribution
154
Distribution
90
Distribution
123
Distribution
167
Distribution


/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.715124) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.731324) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/steve/Software/virtualenv/ccgb/lib/python3.9/site-packages/django/db/models/fields/__init__.py:1448: RuntimeWarning: DateTimeField ComboIssueRequestAgentMap.issue_rised_date received a naive datetime (2022-04-20 08:39:26.746935) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetim

In [11]:
ext = []
for index, row in df.iterrows():
    aggri_officer_ids= [82, 99]
    date_string = row['Date']
    date = pd.to_datetime(date_string).date()
    if  ComboIssueRequest.objects.filter(issue_rised_date=date).exists():
        print(row['Date'])
        ext.append(row['Date'])
print(len(ext))

25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
25-02-2022
26-02-2022
26-02-2022
26-02-2022
26-02-2022
26-02-2022
26-02-2022
26-02-2022
28-02-2022
28-02-2022
02-03-2022
02-03-2022
05-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
07-03-2022
08-03-2022
08-03-2022
08-03-2022
09-03-2022
09-03-2022
09-03-2022
09-03-2022
09-03-2022
09-03-2022
09-03-2022
09-03-2022
10-03-2022
11-03-2022
11-03-2022
11-03-2022
12-03-2022
12-03-2022
12-03-2022
12-03-2022
14-03-2022
15-03-2022
16-03-2022
16-03-2022

In [14]:
ComboIssueRequest.objects.filter(season_id=4, input_combo_id=8).count()

102

In [68]:
# input_list = [139]
import pandas as pd
main_df = pd.DataFrame(list(InputSubStoreInventory.objects.filter(input_store_inventory__input_packet_inventory__input_combo__id__in=[8,9]).order_by('id').values()))
main_df.head(4)
main_df

,id,season_id,input_store_inventory_id,sub_storage_id,section,sub_section,date_of_receipt,quantity_at_receipt,quantity_now,quantity_now_time,unit_id,label_range_start,label_range_end,created_by_id,modified_by_id,time_modified,time_created
0,44,3,56,1,,,2021-05-03,138,0,2021-05-03 21:31:24.444674+00:00,4,0,0,116,116,2021-05-03 21:31:24.449009+00:00,2021-05-03 21:23:26.394901+00:00
1,45,3,57,1,,,2021-05-03,379,0,2021-05-03 21:31:24.453175+00:00,4,0,0,116,116,2021-05-03 21:31:24.457794+00:00,2021-05-03 21:23:27.242839+00:00
2,46,3,58,1,,,2021-05-03,646,0,2021-05-03 21:31:24.462965+00:00,4,0,0,116,116,2021-05-03 21:31:24.467434+00:00,2021-05-03 21:23:29.817577+00:00
3,47,3,59,1,,,2021-05-03,909,0,2021-05-03 21:31:24.471877+00:00,4,0,0,116,116,2021-05-03 21:31:24.475852+00:00,2021-05-03 21:23:34.612385+00:00
4,48,3,60,1,,,2021-05-03,391,0,2021-05-03 21:31:24.480983+00:00,4,0,0,116,116,2021-05-03 21:31:24.485485+00:00,2021-05-03 21:23:41.763420+00:00
5,49,3,61,1,,,2021-05-03,131,0,2021-05-03 21:31:24.489654+00:00,4,0,0,116,116,2021-05-03 21:31:24.494146+00:00,2021-05-03 21:23:45.357718+00:00
6,50,3,62,1,,,2021-05-03,390,0,2021-05-03 21:31:24.498772+00:00,4,0,0,116,116,2021-05-03 21:31:24.503778+00:00,2021-05-03 21:23:46.584898+00:00
7,51,3,63,1,,,2021-05-03,251,0,2021-05-03 21:31:24.507783+00:00,4,0,0,116,116,2021-05-03 21:31:24.511952+00:00,2021-05-03 21:23:50.068127+00:00
8,52,3,64,1,,,2021-05-03,127,0,2021-05-03 21:31:24.516241+00:00,4,0,0,116,116,2021-05-03 21:31:24.521000+00:00,2021-05-03 21:23:52.450120+00:00
9,53,3,65,1,,,2021-05-03,233,0,2021-05-03 21:31:24.525422+00:00,4,0,0,116,116,2021-05-03 21:31:24.530396+00:00,2021-05-03 21:23:53.666013+00:00


In [4]:
input_dict = []

for item in input_list:
    main_df = pd.DataFrame(list(InputSubStoreInventory.objects.filter(input_store_inventory__input_packet_inventory__input_combo__id__in=[8,9]).order_by('id').values()))
    print(item)
    for index, row in main_df.iterrows():
        if item < row['quantity_now'] or item == row['quantity_now']:
            temp_dict = {}
            temp_dict['id'] = [row.id] 
            temp_dict['entered_qty'] = item
            input_dict.append(temp_dict)
            break;
        else:
            temp_dict = {}
            temp_dict['id'] = [row.id] 
            temp_dict['entered_qty'] = row['quantity_now']
            input_dict.append(temp_dict)
            item = item - row['quantity_now']
input_dict

139


[{'id': [43], 'entered_qty': 138}, {'id': [44], 'entered_qty': 1}]

In [3]:
count = 0
for index, row in df.iterrows():
    if UserProfile.objects.filter(mobile=int(row['Moblie Number'])).exists():
        user_ids = UserProfile.objects.filter(mobile=int(row['Moblie Number'])).values_list('user_id')
        if UserClusterMap.objects.filter(user_id__in=user_ids, unique_code=row['Unique Code'], season_id=4):
            agent_id = UserClusterMap.objects.get(user_id__in=user_ids, unique_code=row['Unique Code'], season_id=4).user_id
            count = count + 1
            

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122


In [28]:
user_ids = UserProfile.objects.filter(user_type_id=5).values_list('user_id', flat=True)
UserClusterMap.objects.filter(user_id__in=user_ids, season_id=4).values_list('user__username', flat=True)

<QuerySet ['srinivasa', 'harish', 'naveen', 'prasad', 'siddapa', 'madappa', 'kiran', 'siddaraju', 'shivakumar0', 'Kollegal area0', 'Hassan0', 'Dudda0']>

In [30]:
for index, row in df.iterrows():
    if not User.objects.filter(username=row['Supervisor Name']).exists():
        print(row['Supervisor Name'])

shivakumar00
Srinivasa G
shivakumar00
Srinivasa G
Srinivasa G
Srinivasa G
Srinivasa G
Srinivasa G
Srinivasa G
Srinivasa G
Srinivasa G
0
0
0
0
0
0
shivakumar00
shivakumar00
Siddaraju
Naveena
Madappa B
Madappa B
Siddaraju
Siddaraju
Harish
Harish
Kiran K B
Harish
Harish
Siddaraju
Kiran K B
Kiran K B
Kiran K B
Madappa B
Harish
Kiran K B
Siddaraju
Naveena
Naveena
Naveena
Naveena
Harish
Harish
Naveena
Harish
Kiran K B
Madappa B
Harish
Siddaraju
Siddaraju
0
0
Kiran K B
Naveena
Naveena
Naveena
Harish
Kiran K B
shivakumar00
Naveena
Harish
Madappa B
Kiran K B
Naveena
Naveena
Naveena
Harish
Siddaraju
shivakumar00
Siddaraju
shivakumar00
Srinivasa G
Srinivasa G
Srinivasa G
shivakumar00
shivakumar00
Srinivasa G
Siddaraju
Madappa B
Naveena
Srinivasa G
Madappa B
Naveena
Naveena
shivakumar00
0
0
0
0
0
0
